In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, destroy,
                   expect, fock, fock_dm, mesolve, qeye, sigmax, sigmay,
                   sigmaz, tensor, thermal_dm, ket2dm, ptrace, qdiags)
import qutip
import time
from scipy import fft
import itertools
from itertools import permutations
from types import SimpleNamespace
from latticefunctions import (purity, purity_rho1, von_neumann_entropy, von_neumann_entropy_rho1, minmax_eigvals, eigenstates0, 
                                multipart_tensor, var_op, result_dict, permute_particles, simulate, plots, secondary_observables)

smesolve = qutip.stochastic.smesolve
ssesolve = qutip.stochastic.ssesolve

In [2]:
#System parameters
L = 7                   #Number of sites
eigs = eigenstates0(L)  #eigenstates for 0 potential case
ct = int(L/2)           #central site
periodic_boundary = False

#Time
T = 100
N_times = T*80

#Hamiltonian parameters
J = 1           #tunneling rate
U0 = 0          #1-particle potential
V0 = 2          #2-particle potential
U_site = ct    #site of the 1-particle potential (U)

#Measurement parameters
gamma = 0         #Decoherence rate
gamma_m = 0.4     #Measurement strength
probed_n = ct     #Probed site
meas_start = 0
meas_stop = None

#Iterations
N_traj = 100
N_runs = 1     #Number of simulation runs

In [3]:
# Eigenstates combination for generic N-particle state
def eig_st(*indices):
    return tensor(*[eigs[i] for i in indices])

# Site basis combination for generic N-particle Fock state
def site_st(*indices):
    return tensor(*[fock(L, i) for i in indices])

In [4]:
#Initial state (can be both a state vector or a density operator; can be given as combination of Fock states or energy eigenstates)
rho0 = eig_st(0,6)
rho0 = rho0/rho0.norm()

In [5]:
#Number of particles
N = len(rho0.dims[0])

#Time list
tlist = np.linspace(0,T,N_times)

#HAMILTONIAN
#Transition operators for the two atoms, +/- 1 position
tp_ops = [fock(L,i+1)*fock(L,i).dag() for i in range(L-1)]
if periodic_boundary==True:
    tp_ops += [fock(L,0)*fock(L,L-1).dag()]

tp = multipart_tensor(sum(tp_ops), L, N)
tm = tp.dag()
K = -J*(tp+tm)      #Kinetic part for the two atoms


#On-site 1-particle potential
U = U0*multipart_tensor(fock_dm(L,U_site), L, N)

#Repulsive 2-particle potential
V_sites = [V0*multipart_tensor(fock_dm(L,i), L, N, order=2) for i in range(L)]
V = sum(V_sites)

H = K+U+V

In [6]:
#Some useful operators
#Occupation of the probed site
def site_occ(l):
    return multipart_tensor(fock_dm(L,l), L, N)
occ_probed = site_occ(probed_n)


def parity_op(L, N, n):
    ops = [qeye(L)]*N
    parity = sum([fock(L,i)*fock(L,(L-1)-i).dag() for i in range(L)])
    ops[n-1] = parity
    return tensor(*ops)

def antiparity(L,N):
    parity = sum([1j*(fock(L,ct-i)*fock(L,ct+i).dag() - fock(L,ct+i)*fock(L,ct-i).dag()) for i in range(int(L/2))])
    return multipart_tensor(parity, L, N)


#Permutation operators
def perm_op(*idx):
    l=L
    return permute_particles(l, *idx)

perm = permutations(range(N))
if N==3:
    boson_sub = (1/6)*sum([perm_op(i) for i in perm])
    fermion_sub = (1/6)*(perm_op(1,2,3) + perm_op(2,3,1) + perm_op(3,1,2) - perm_op(3,2,1) - perm_op(2,1,3) - perm_op(1,3,2))
    immanon_sub = (1/3)*(2*perm_op(1,2,3) - perm_op(2,3,1) - perm_op(3,1,2))



#Current operator from site m to n
def j_mn(m, n):
    return multipart_tensor(1j*J*(fock(L,m)*fock(L,n).dag() - fock(L,n)*fock(L,m).dag()), L, N)
#Current away from site
def j_m(m):
    return j_mn(m,m+1) + j_mn(m,m-1)
    
#Current away from the center
j_c = j_m(ct)

In [7]:
#Dissipation
c_opss = [np.sqrt(gamma)*occ_probed]
#c_opss = [np.sqrt(gamma)*site_occ(i) for i in range(L)]

#Probing
sc_ops = [np.sqrt(gamma_m)*occ_probed]
#sc_ops = [np.sqrt(gamma_m)*multipart_tensor(fock_dm(L,ct), L, N, order=2)]
#sc_ops = [np.sqrt(gamma_m)*antiparity(L,N)]

#Measurement window
if meas_stop==None:
    meas_stop = tlist[-1]
meas_coeffs = np.array([1.0 if meas_start<t<=meas_stop else 0.0 for t in tlist])

#Applies the measurement coefficients to each sc_opss
sc_opss = [[op, meas_coeffs] for op in sc_ops]

In [8]:
def build_eops(L, N, eigs):
    e_ops = []
    keys = {}

    id_op = [qeye(L)]*N
    
    #Site occupations
    for which in range(1,N+1):
        for i in range(L):
            keys.setdefault(f"P_fock{which}", []).append(len(e_ops))    #Creates list of site occupations for each particle
            op = list(id_op)
            op[which-1] = fock_dm(L,i)
            e_ops.append(tensor(*op))

    #Eigenstates
    for which in range(1,N+1):
        for i in range(L):
            keys.setdefault(f"P_eig{which}", []).append(len(e_ops))
            op = list(id_op)
            op[which-1] = eigs[i] * eigs[i].dag()
            e_ops.append(tensor(*op))

    #Parity operator
    for n in range(1,N+1):
        keys[f"Parity_op{n}"] = len(e_ops); e_ops.append(parity_op(L,N,n))
    
    keys["occ_probed"] = len(e_ops); e_ops.append(occ_probed)
    keys["entropy"] = len(e_ops); e_ops.append(von_neumann_entropy)
    keys["entropy_rho1"] = len(e_ops); e_ops.append(von_neumann_entropy_rho1)
    keys["ct_current"] = len(e_ops); e_ops.append(j_c)
    keys["ct_current_var"] = len(e_ops); e_ops.append(var_op(j_c))
    if N==2:
        keys["p_exc"] = len(e_ops); e_ops.append(perm_op(2,1))
    if N==3:
        keys["immanon_sub"] = len(e_ops); e_ops.append(immanon_sub)
        keys["boson_sub"] = len(e_ops); e_ops.append(boson_sub)
        keys["fermion_sub"] = len(e_ops); e_ops.append(fermion_sub)
        
    return e_ops, keys

In [9]:
e_opss, keys = build_eops(L, N, eigs)

In [10]:
#print(e_opss[0].dims, c_opss[0].dims, rho0.dims, H.dims)

In [11]:
for i in range(N_runs):
    #Seed for consistency. If set as None, it takes by default int(time.time()*10)
    seed = None
    
    #Simulation with mesolve/smesolve/ssesolve depending on gamma and gamma_m
    result = simulate(H, rho0, tlist, e_opss, c_opss, sc_opss, gamma, gamma_m, N_traj, seed)
    
    #Making a result as dictionary to avoid calling it as: result.expect[keys["P_fock1"]], but more simply as res["P_fock1"]
    res = result_dict(result, keys)

    #Adding extra observables that can be derived from the primary ones
    res = secondary_observables(res,N,L)
    
    """Plot options: "position", "sites_occupations", "eigen_probs", "probed_site_particles", "probed_site_occ",
		 "energy", "parity", "parity_op", "entropy_tot", "entropy_rho1", "bosonicness", 
		 "symmetries", "occ_probed", "center_current", "center_current_var"
         All of them: "everything"
    """
    
    plots(tlist, res, N, L, probed_n, draw=["parity", "eigen_probs"])

10.0%. Run time:  45.88s. Est. time left: 00:00:06:52
20.0%. Run time:  98.44s. Est. time left: 00:00:06:33
30.0%. Run time: 151.52s. Est. time left: 00:00:05:53
40.0%. Run time: 205.16s. Est. time left: 00:00:05:07
50.0%. Run time: 258.97s. Est. time left: 00:00:04:18
60.0%. Run time: 312.82s. Est. time left: 00:00:03:28
70.0%. Run time: 366.72s. Est. time left: 00:00:02:37
80.0%. Run time: 421.46s. Est. time left: 00:00:01:45
90.0%. Run time: 475.61s. Est. time left: 00:00:00:52
100.0%. Run time: 529.66s. Est. time left: 00:00:00:00
Total run time: 535.11s
